In [ ]:
import numpy as np
import cv2

import matplotlib.pyplot as plt
from tqdm.auto import tqdm, trange

from solver import (
    bird_eye_view,
    find_duck,
    find_boarder_line,
    find_dotted_line,
    filter_lines,
    get_average_angle,
    get_average_pos
)

In [ ]:
def load_video(path):
    cap = cv2.VideoCapture(path)
    
    frames = []
    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret == True:
            frames += [frame]
        else:
            break
            
    cap.release()
    
    return frames

In [ ]:
def save_video(path, imgs):
    h, w = imgs[0].shape[:2]
    
    out = cv2.VideoWriter(path, cv2.VideoWriter_fourcc('M','J','P','G'), 25, (w, h))

    for frame in imgs:
        out.write(frame)

    out.release()

In [ ]:
def proc_video(imgs, foo):
    out = []
    for img in tqdm(imgs):
        img = foo(img)
        
        if isinstance(img, tuple):
            img = list(img)
        else:
            img = [img]
        
        for i in range(len(img)):
            if len(img[i].shape) == 2:
                img[i] = cv2.cvtColor(img[i], cv2.COLOR_GRAY2BGR)
                
        hs = [img_i.shape[0] for img_i in img]
        max_h = max(hs)
        
        for i in range(len(img)):
            h, w = img[i].shape[:2]
            new_img = np.zeros((max_h, w, 3), dtype=np.uint8)
            new_img[:h] = img[i]
            img[i] = new_img
            
        out += [np.concatenate(img, axis=1)]
        
    return out

In [ ]:
def crop_image(img):
    h, w = img.shape[:2]
    return img[h//2:, :w//2]

In [ ]:
def draw_line(img, line, color=(0, 0, 255)):
    if line is None:
        return
    
    a, b, c = line
    
    h, w = img.shape[:2]
    
    y1 = 0
    y2 = h
    
    x1 = (-c - b * y1) / a
    x2 = (-c - b * y2) / a
    x1, x2 = int(x1), int(x2)
    
    cv2.line(img, (x1, y1), (x2, y2), color, 3)

In [ ]:
def draw_bb(img, bb, color=(0, 255, 0)):
    if bb is None:
        return

    cv2.rectangle(img, (bb[0], bb[1]), (bb[2], bb[3]), color, 3)

In [ ]:
def draw_robot(img, center, color=(255, 0, 0)):
    x, y = center
    x, y = int(x), int(y)
    cv2.line(img, (x, y), (x, y - 30), color, 3)
    cv2.circle(img, (x, y), 10, color, -1)

In [ ]:
def move_image(img, center, angle, pos):
    if angle is None:
        return np.zeros_like(img, dtype=np.uint8)
    
    rot_mat = cv2.getRotationMatrix2D(tuple(center), angle / np.pi * 180, 1.0)
    result = cv2.warpAffine(img, rot_mat, img.shape[1::-1], flags=cv2.INTER_LINEAR)
    
    if pos is None:
        return result
    
    h, w = result.shape[:2]
    pos = int(pos)
    if pos > 0:
        result[:, pos:] = result[:, :-pos]
        result[:, :pos] = 0
    elif pos < 0:
        result[:, :pos] = result[:, -pos:]
        result[:, pos:] = 0
    
    return result

In [ ]:
def proc_img(img):
    img = crop_image(img)
    biw = bird_eye_view(img)
    
    duck_cmp, duck_bb = find_duck(biw)
    
    boarder_line_mask, boarder_lines = find_boarder_line(biw)
    dotted_line_mask, dotted_line = find_dotted_line(biw)
    
    h, w = biw.shape[:2]
    center = np.array([w / 2, h])
    
    boarder_lines, dotted_line = filter_lines(boarder_lines, dotted_line)
    angle = get_average_angle(boarder_lines, dotted_line)
    pos = get_average_pos(boarder_lines, dotted_line, center)
    
    draw_line(biw, boarder_lines[0], color=(0, 0, 255))
    draw_line(biw, boarder_lines[1], color=(0, 0, 255))
    draw_line(biw, dotted_line, color=(0, 255, 0))
    
    draw_bb(biw, duck_bb)
    
    draw_robot(biw, center)
    
    biw_moved = move_image(biw, center, angle, pos)
    
    return img, biw, biw_moved

In [ ]:
imgs = load_video("record.mp4")
imgs = proc_video(imgs, proc_img)
save_video("res.mp4", imgs)